In [1]:
import matplotlib.pyplot as plt
import glob
from astropy.io import fits
from astropy.visualization import make_lupton_rgb
import numpy as np
import scipy.ndimage as snd
from scipy import optimize



In [ ]:
def find_skylevel(filt, file_path, plot=True):
    """ 
    Measures background noise
    """
    backnoise=[]
    numpix_list, aduval_list = [], []
    
    numpix, aduvals = np.histogram(fits.getdata(science_red_files_all[i]).flatten(), 
                                           bins=1000, range=[-1e3,5e3])
            skylevel = aduvals[where(numpix==numpix.max())][0]
            skylevels.append(skylevel)
            numpix_list.append(numpix)
            aduval_list.append(aduvals)
            
    if plot:
        plt.figure(figsize=(18,9))
        for i in range(len(skylevels)):
            plt.plot(aduval_list[i][:-1], numpix_list[i],
                     label=str(science_red_files_filt[i].split('/')[-1]))
            plt.axvline(skylevels[i], color='k')
        plt.xlabel('pix val',fontsize=20)
        plt.ylabel('number of pixels',fontsize=20)
        plt.yscale('log')
        plt.legend()
        plt.title('Histogram of reduced '+filt+' frame pixel values',fontsize=20)
            
    return skylevels, science_red_files_filt

In [4]:
def find_object_centroids_filterbysize(img, threshold, minsize):
    """
    inputs: 
    img --> 2D array 
    threshold --> single value
    minsize --> minimum object size
    """

    labels, num = snd.label(img > threshold, np.ones((3,3)))     # scipy labels/segments the image using a threshold
    centers = snd.center_of_mass(img, labels, range(1,num+1))    # scipy calculates the center of mass on the labeled img
    #print(centers)
    x = array(centers)[:,1]
    y = array(centers)[:,0]
    slices = snd.find_objects(labels)
    xs = np.array([objlabel[1].stop-objlabel[1].start for objlabel in slices])  # takes the min and max label slices
    ys = np.array([objlabel[0].stop-objlabel[0].start for objlabel in slices])  #  to find a rough object size

    maxsize = 1025    # I hardcoded this in so that some spurious objects would be skipped. Change/delete if you like
    bigenough = np.where((xs>minsize) & (ys>minsize) & (xs<maxsize) & (ys<maxsize))
    xc,yc = x[bigenough],y[bigenough]
    xs,ys = xs[bigenough],ys[bigenough]
    
    print(str(len(xc))+' objects found')
    return xc,yc,xs,ys
